In [1]:


#from __future__ import absolute_import, division, print_function, unicode_literals

#!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

'''
# Download the file
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)
'''

# ANKI corpus
#path_to_file = "dan.txt"
path_to_file = "dan2.txt"


# European Parl corpus - 1 file
#path_to_file = "Tester_v4.txt"

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-
    #spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:4000]] # <--- IMPORT!!

    return zip(*word_pairs)
  
en, sp = create_dataset(path_to_file, None)
print(en[-1])
print(sp[-1])
  
en_sentence = u"May I borrow this book?"
sp_sentence = u"Maa jeg laane denne bog?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

def max_length(tensor):
    return max(len(t) for t in tensor)
  
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer
  
# Try experimenting with the size of that dataset
num_examples = 4000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))
    
print('Beginning finished')

<start> the next item is the debate on the talks in austria on forming a government . <end>
<start> naeste punkt paa dagsordenen er debatten om forhandlingerne i oestrig med henblik paa regeringsdannelsen . <end>
<start> may i borrow this book ? <end>
b'<start> maa jeg laane denne bog ? <end>'
Beginning finished


In [2]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape



(TensorShape([64, 146]), TensorShape([64, 143]))

In [3]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))
  
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 146, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [4]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, hidden_size)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights
  
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 146, 1)


In [5]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights
  
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

Decoder output shape: (batch_size, vocab size) (64, 6895)


In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.6717
Epoch 1 Loss 1.2800
Time taken for 1 epoch 17668.852115154266 sec

Epoch 2 Batch 0 Loss 1.3412
Epoch 2 Loss 1.1679
Time taken for 1 epoch 17049.76627278328 sec

Epoch 3 Batch 0 Loss 1.0249
Epoch 3 Loss 1.1063
Time taken for 1 epoch 17107.988303899765 sec

Epoch 4 Batch 0 Loss 1.1607
Epoch 4 Loss 1.0549
Time taken for 1 epoch 17141.632566690445 sec

Epoch 5 Batch 0 Loss 0.9949
Epoch 5 Loss 1.0028
Time taken for 1 epoch 17164.596872329712 sec

Epoch 6 Batch 0 Loss 1.1123
Epoch 6 Loss 0.9602
Time taken for 1 epoch 17233.823779582977 sec

Epoch 7 Batch 0 Loss 0.9926
Epoch 7 Loss 0.9234
Time taken for 1 epoch 17222.64224100113 sec

Epoch 8 Batch 0 Loss 0.8448
Epoch 8 Loss 0.8904
Time taken for 1 epoch 17270.080790519714 sec

Epoch 9 Batch 0 Loss 0.8978
Epoch 9 Loss 0.8585
Time taken for 1 epoch 17219.095610141754 sec

Epoch 10 Batch 0 Loss 0.8126


In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot
  
  # function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()
    
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

print('Functions loaded')

In [ ]:
#print('Input a Danish Word')
y = ''
y = input('Input a Danish Word: ')
translate(u''+y)

My older version

In [ ]:
def enc_dec_testing(n_input, n_output, vocab_uk, vocab_dk, optimizer, loss_unit, n_units):

#=============== ENCODER =================
    encoder_inputs = Input(shape=(None, n_input), name='Encoder_Input')
  
    encod_embed = Embedding(output_dim=n_input, input_dim=vocab_uk, name='Encod_Embed')
    encd_embed_out = encod_embed(encoder_inputs)
  
    lstm1 = LSTM(n_units, return_state = True, return_sequences=True, name='LSTM_1')
    lstm1_x, lstm1_h, lstm1_c = lstm1(encd_embed_out)
    lstm1_states = [lstm1_h, lstm1_c]

    #encoder_input_l2 = Input(shape=(n_input,1), name='Encoder_Input_l2')
    
    lstm2 = LSTM(n_input, return_state = True, return_sequences=True, name='LSTM_2')
    lstm2_x, lstm2_h, lstm2_c = lstm2(lstm1_x, initial_state=lstm1_states)
    lstm2_states = [lstm2_h, lstm2_c]
    
    '''
    encoder_input2 = Input(shape=(n_input,), name='Encoder_Input2')
    encod_embed2 = Embedding(output_dim=512, input_dim=vocab_uk, name='Encod_Embed')
    encd_embed_out2 = encod_embed2(encoder_input2)
    lstm2 = LSTM(n_units, return_state = True, return_sequences=True, name='LSTM_2')
    lstm2_x, lstm2_h, lstm2_c = lstm2(encd_embed_out2, initial_state=lstm1_states)
    lstm2_states = [lstm2_h, lstm2_c]
    '''
    
    encoder_output = Dense(1, activation='softmax', name='Encoder_Dense')
    encoder_out_dense = encoder_output(lstm2_x)
 #=============== Encoder model =================   
    encoder_model = Model([encoder_inputs] + lstm2_states)


#=============== DECODER =================
    decoder_inputs = Input(shape=(None,n_input), name='Decoder_Input')
  
    dec_embed = Embedding(output_dim=vocab_dk, input_dim=106, name='Dec_Embedder')
    dec_embed_out = dec_embed(decoder_inputs)

    lstm3 = LSTM(n_output, return_state = True, return_sequences=True, name='LSTM_3')
    lstm3_x, lstm3_h, lstm3_c = lstm3(dec_embed_out, initial_state=lstm2_states)
    lstm3_states = [lstm3_h, lstm3_c]
    
    lstm4 = LSTM(n_output, return_state = True, return_sequences=True, name='LSTM_4')
    lstm4_x, lstm4_h, lstm4_c = lstm4(lstm3_x, initial_state=lstm3_states)
    lstm4_states = [lstm4_h, lstm4_c]
    
    decoder_output = Dense(1, activation='softmax', name='Decoder_Dense')
    decoder_out_dense = decoder_output(lstm4_x)
    
    
#=============== Model out =================
    #model_test = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[lstm2_x,lstm4_x])
    #model_test = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[decoder_dense_out])
    
    # Seems like there is a correlation between Model and Fit <- they must apparently be the same.
    model_test = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[encoder_out_dense, decoder_out_dense])

#============================================
    model_test.compile(optimizer=optimizer, loss=loss_unit, metrics=['acc']) # <- WORKS!!!!
    
    #model_test.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
    model_test.summary()
    plot_model(model_test, show_shapes=True, to_file='Encoder_Decoder_testing.png')
#============================================    
    return model_test, encoder_model
#============================================

print('Model build finished')

In [ ]:
print('X_train.shape: ',X_train.shape)
print('y_train.shape: ',y_train.shape)

n_steps = X_train.shape[1]

print('n_steps: ', n_steps)
n_units = 256
#loss_unit = 'categorical_crossentropy'
#loss_unit = 'binary_crossentropy'
#loss_unit = 'logcosh'
loss_unit = 'mean_squared_error'
#loss_unit = 'crossentropy'


#============================================
optimizer = 'adam'
train_test, encod_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([X_train, y_train], [X_test, y_test], verbose=1, batch_size=4)
print('Score before training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score before training: ', train_test.metrics_names, score)
#============================================

#score = encod_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
#print('Score before training: ', encod_test.metrics_names, score)

#============================================

'''
es = EarlyStopping(monitor='val_loss', mode='min')
train_test.fit([X_train, y_train], [X_test, y_test], batch_size=4, epochs=50, verbose=0, callbacks=[es])
score = train_test.evaluate([X_train, y_train], [X_test, y_test], verbose=1)
print('Score after training: \n', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: \n', train_test.metrics_names, score)
'''

In [ ]:
#========== TESTING PREDICTION ================
# So far so good ... now how to Predict and get out the stuff that I want?

#print('Word list UK: ', english_fit2text.word_index)
#print('Word list DK: ', danish_fit2text.word_index)

print('='*100)
text_testing_dk = ['der var', 'stilhed']
text_testing_uk = ['there was', 'silence']
#text_testing.split()
#print(text_testing)
print('der plads: ', danish_fit2text.word_index['der'])
print('there plads: ', english_fit2text.word_index['there'])
print('='*100)

testing_txt2seq_dk = danish_fit2text.texts_to_sequences(text_testing_dk)
testing_txt2seq_uk = english_fit2text.texts_to_sequences(text_testing_uk)

#print('testing_txt2seq dk: ', np.transpose(testing_txt2seq_dk))
#print('testing_txt2seq_uk: ', testing_txt2seq_uk)
print('='*100)

testing_seq2txt = danish_fit2text.sequences_to_texts(testing_txt2seq_dk)
testing_seq2txt_uk = english_fit2text.sequences_to_texts(testing_txt2seq_uk)
print('testing_seq2txt dk: ', testing_seq2txt)
print('testing_seq2txt uk: ', testing_seq2txt_uk)
print('='*100)

max_sequence_length = 106

testing_txt2seq_pad_dk = pad(testing_txt2seq_dk, max_sequence_length)
testing_txt2seq_pad_uk = pad(testing_txt2seq_uk, max_sequence_length)

print('testing_txt2seq_pad dk: ', testing_txt2seq_pad_dk)
print('testing_txt2seq_pad uk: ', testing_txt2seq_pad_uk)
print('='*100)
print('Shape dk: ', testing_txt2seq_pad_dk.shape)
print('Shape uk: ', testing_txt2seq_pad_uk.shape)
#testing_txt2seq_reshaped = testing_txt2seq_pad.reshape((-1, testing_txt2seq_pad[-1]))
print('='*100)

#testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(*testing_txt2seq_pad_dk.shape, 1)
#testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(*testing_txt2seq_pad_uk.shape, 1)


testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(2,106,1)
testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(2,106,1)


print('testing_txt2seq_reshaped: ', testing_txt2seq_reshaped_dk.shape)
print('testing_txt2seq_reshaped_uk: ', testing_txt2seq_reshaped_uk.shape)
print('='*100)

#======================================================================================
dims2 = train_test.predict([testing_txt2seq_reshaped_uk, testing_txt2seq_reshaped_dk]) #<---- What is the input UK , DK ???
print(dims2)

'''
import tensorflow as tf
predict1 = tf.argmax(dims2[0])
print('Predict 1:\n', predict1)
'''
#print('dims2.max', dims2.max())
#print('dims2.min', dims2.min())
#print('dims2.mean', dims2.mean())
#print('dims2.ndim', dims2.ndim)
#print('dims2.mean', dims2.min())
#print('dims2.mean', dims2.shape)

print('='*100)
arrayout = np.argmax(dims2)
print('Array out:', arrayout)
print('='*100)
prediction_dk = np.argmax(dims2, axis = 0)
prediction_uk = np.argmax(dims2, axis = 1)

print('prediction_dk: ',prediction_dk)
print('prediction_uk: ', prediction_uk)

It could be fun to run test on this model about this LOSS and ACTIVATION function works best...

In [ ]:
n_steps = uk_training_reshape.shape[1]
print('n_steps: ', n_steps)
n_units = 256
#loss_unit = 'categorical_crossentropy'
loss_unit = 'binary_crossentropy'

#============================================
optimizer = 'adam'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'sgd'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================

optimizer = 'rmsprop'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'adagrad'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'adadelta'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'adamax'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================
optimizer = 'nadam'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)
#============================================


#================= TESTING ================================
'''
train_test = enc_dec_testing(
    uk_training_reshape.shape,
    dk_training_reshape.shape[1],
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    n_units)
'''


print('='*200)
print('Finished with NN design')
print('='*200)



In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min')
train_test.fit([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], epochs=50, verbose=0, callbacks=[es])

optimizer = 'nadam'
train_test = enc_dec_testing(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    optimizer,
    loss_unit,
    n_units)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)
print('Score after training: ', train_test.metrics_names, score)
print('='*200)
print('optimizer: ', optimizer)
print('Score after training: ', train_test.metrics_names, score)

In [ ]:
#================== ======================================== ==================
print('Understand your data for far:')
print('english_vocab_size: ', uk_vocab_size)
print('danish_vocab_size: ', dk_vocab_size)
print('dk_training_reshape.shape DK: ', dk_training_reshape.shape)
print('uk_training_reshape.shape UK:', uk_training_reshape.shape)
print('dk_test_reshape.shape DK: ', dk_test_reshape.shape)
print('uk_test_reshape.shape UK:', uk_test_reshape.shape)
#================== ======================================== ==================

#train_test.fit(uk_training_reshape, dk_training_reshape, validation_data=(X_test, y_test), epochs=5, verbose=1)
es = EarlyStopping(monitor='val_loss', mode='min')
train_test.fit([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], epochs=50, verbose=0, callbacks=[es]) #<- works!!!!!
#train_test.fit(uk_training_reshape, dk_training_reshape, [uk_test_reshape, dk_test_reshape], epochs=50, verbose=1, callbacks=[es])
#train_test.fit([X_train, y_train], [X_test, y_test], epochs=50, verbose=0, callbacks=[es])
#train_test.fit([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], epochs=50, verbose=0, callbacks=[es])

#train_test.fit([uk_training_reshape, dk_training_reshape], validation_data=(X_test, y_test), epochs=5, verbose=1)
#train_test.fit([uk_training_reshape, dk_training_reshape], dk_test_reshape, epochs=5, verbose=1)
#train_test.fit([uk_training_reshape, dk_training_reshape], epochs=5, verbose=1)
#train_test.fit([X_train, y_train], [X_test, y_test], epochs=5, verbose=1)
#train_test.fit([X_train, y_train], [uk_test_reshape, dk_test_reshape], epochs=5, verbose=1, validation_split=0.2)
#train_test.fit_generator()


# But our Eval must be based on our Test data -> how will it respond?
#score = train_test.evaluate([uk_test_reshape, dk_test_reshape], dk_test_reshape, verbose=1)
#score = train_test.evaluate(uk_test_reshape, dk_test_reshape, epochs=5, verbose=1)
score = train_test.evaluate([uk_training_reshape, dk_training_reshape], [uk_test_reshape, dk_test_reshape], verbose=1)

print('Score after training: ', train_test.metrics_names, score)


In [ ]:
#========== TESTING PREDICTION ================
# So far so good ... now how to Predict and get out the stuff that I want?

#print('Word list UK: ', english_fit2text.word_index)
#print('Word list DK: ', danish_fit2text.word_index)

print('='*100)
text_testing_dk = ['der var', 'stilhed']
text_testing_uk = ['there was', 'silence']
#text_testing.split()
#print(text_testing)
print('der plads: ', danish_fit2text.word_index['der'])
print('there plads: ', english_fit2text.word_index['there'])
print('='*100)

testing_txt2seq_dk = danish_fit2text.texts_to_sequences(text_testing_dk)
testing_txt2seq_uk = english_fit2text.texts_to_sequences(text_testing_uk)

#print('testing_txt2seq dk: ', np.transpose(testing_txt2seq_dk))
#print('testing_txt2seq_uk: ', testing_txt2seq_uk)
print('='*100)

testing_seq2txt = danish_fit2text.sequences_to_texts(testing_txt2seq_dk)
testing_seq2txt_uk = english_fit2text.sequences_to_texts(testing_txt2seq_uk)
print('testing_seq2txt dk: ', testing_seq2txt)
print('testing_seq2txt uk: ', testing_seq2txt_uk)
print('='*100)

max_sequence_length = 106

testing_txt2seq_pad_dk = pad(testing_txt2seq_dk, max_sequence_length)
testing_txt2seq_pad_uk = pad(testing_txt2seq_uk, max_sequence_length)

print('testing_txt2seq_pad dk: ', testing_txt2seq_pad_dk)
print('testing_txt2seq_pad uk: ', testing_txt2seq_pad_uk)
print('='*100)
print('Shape dk: ', testing_txt2seq_pad_dk.shape)
print('Shape uk: ', testing_txt2seq_pad_uk.shape)
#testing_txt2seq_reshaped = testing_txt2seq_pad.reshape((-1, testing_txt2seq_pad[-1]))
print('='*100)

#testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(*testing_txt2seq_pad_dk.shape, 1)
#testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(*testing_txt2seq_pad_uk.shape, 1)


testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(1,2,106)
testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(1,2,106)


print('testing_txt2seq_reshaped: ', testing_txt2seq_reshaped_dk.shape)
print('testing_txt2seq_reshaped_uk: ', testing_txt2seq_reshaped_uk.shape)
print('='*100)

#======================================================================================
dims2 = train_test.predict([testing_txt2seq_reshaped_uk, testing_txt2seq_reshaped_dk]) #<---- What is the input UK , DK ???
#print(dims2)
import tensorflow as tf
predict1 = tf.argmax(dims2[0])
print('Predict 1:\n', predict1)

#print('dims2.max', dims2.max())
#print('dims2.min', dims2.min())
#print('dims2.mean', dims2.mean())
#print('dims2.ndim', dims2.ndim)
#print('dims2.mean', dims2.min())
#print('dims2.mean', dims2.shape)

print('='*100)
arrayout = np.argmax(dims2)
print('Array out:', arrayout)
print('='*100)
prediction_dk = np.argmax(dims2, axis = 0)
prediction_uk = np.argmax(dims2, axis = 1)

print('prediction_dk: ',prediction_dk)
print('prediction_uk: ', prediction_uk)

In [ ]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
  # define training encoder
  encoder_inputs = Input(shape=(106,1))
  encoder = LSTM(n_units, return_state=True)
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)
  encoder_states = [state_h, state_c]
   
  # define training decoder
  decoder_inputs = Input(shape=(106,1))
  decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  decoder_dense = Dense(1, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  # define inference encoder
  encoder_model = Model(encoder_inputs, encoder_states)
  # define inference decoder
  decoder_state_input_h = Input(shape=(n_units,))
  decoder_state_input_c = Input(shape=(n_units,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  
  decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
  decoder_states = [state_h, state_c]
  decoder_outputs = decoder_dense(decoder_outputs)
  
  decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
  # return all models
  
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
  model.summary()
  plot_model(model, show_shapes=True, to_file='Encoder_Decoder_testing_vNet.png')
    
  return model, encoder_model, decoder_model

n_steps = uk_training_reshape.shape[1]
print('n_steps: ', n_steps)
n_units = 256

#train, infenc, infdec = define_models(n_features, n_features, 128)

train, infenc, infdec = define_models(
    n_steps,
    n_steps,
    n_units)

#train.fit([X1, X2], y, epochs=1)

train.fit([uk_training_reshape, dk_training_reshape], uk_test_reshape, epochs=5, verbose=1, validation_split=0.2)

In [ ]:
#===============================================================================

def enc_dec_alternative(n_input, n_output, vocab_uk, vocab_dk, n_units):
  
	# define training encoder
  encoder_inputs = Input(shape=(n_input,1), name='Encoder_Input')
	
  encoder = LSTM(n_units, return_state=True, name='Encoder_lstm') 
  encoder_outputs, state_h, state_c = encoder( encoder_inputs )
  encoder_states = [state_h, state_c] 
  #=============================================================================
	# define training decoder
  decoder_inputs = Input(shape=(n_output,1), name='Decoder_Input')
	
  decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True, name='Decoder_lstm')
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	
  decoder_dense = Dense(n_output, shape=(106,1), activation='softmax', name='Decoder_Dense')
  decoder_outputs = decoder_dense(decoder_outputs)
	#=============================================================================
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  
	# define inference encoder
  encoder_model = Model(encoder_inputs, encoder_states)
  
  #=============================================================================
	# define inference decoder
  decoder_state_input_h = Input(shape=(None,n_units))
  decoder_state_input_c = Input(shape=(None,n_units))
  
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  
  decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
  
  decoder_states = [state_h, state_c]
  
  decoder_outputs = decoder_dense(decoder_outputs)
  
  decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
  
  #===============================================================================
	# return all models
  return model, encoder_model, decoder_model

#train, infenc, infdec = define_models(n_features, n_features, 128)
#train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

n_steps = uk_training_reshape.shape[1]
n_units = 256


train, infec, infdec = enc_dec_alternative(
    n_steps,
    n_steps,
    uk_vocab_size + 1,
    dk_vocab_size + 1,
    n_units)

train.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
train.summary()
plot_model(train, show_shapes=True, to_file='Encoder_Decoder.png')

In [ ]:
# Fitting must be based on our Training data
train.fit([uk_training_reshape, dk_training_reshape], dk_training_reshape, 256, epochs=5, verbose=1)

# But our Eval must be based on our Test data -> how will it respond?
score = train.evaluate([uk_test_reshape, dk_test_reshape], dk_test_reshape, verbose=1)

print('Score after training: ', encdec_model_dev.metrics_names, score)

In [ ]:
# So far so good ... now how to Predict and get out the stuff that I want?

#print('Word list UK: ', english_fit2text.word_index)
#print('Word list DK: ', danish_fit2text.word_index)

print('='*100)
text_testing_dk = ['der var', 'stilhed']
text_testing_uk = ['there was', 'silence']
#text_testing.split()
#print(text_testing)
print('der plads: ', danish_fit2text.word_index['der'])
print('there plads: ', english_fit2text.word_index['there'])
print('='*100)

testing_txt2seq_dk = danish_fit2text.texts_to_sequences(text_testing_dk)
testing_txt2seq_uk = english_fit2text.texts_to_sequences(text_testing_uk)

#print('testing_txt2seq dk: ', np.transpose(testing_txt2seq_dk))
#print('testing_txt2seq_uk: ', testing_txt2seq_uk)
print('='*100)

testing_seq2txt = danish_fit2text.sequences_to_texts(testing_txt2seq_dk)
testing_seq2txt_uk = english_fit2text.sequences_to_texts(testing_txt2seq_uk)
print('testing_seq2txt dk: ', testing_seq2txt)
print('testing_seq2txt uk: ', testing_seq2txt_uk)
print('='*100)

max_sequence_length = 106

testing_txt2seq_pad_dk = pad(testing_txt2seq_dk, max_sequence_length)
testing_txt2seq_pad_uk = pad(testing_txt2seq_uk, max_sequence_length)

print('testing_txt2seq_pad dk: ', testing_txt2seq_pad_dk)
print('testing_txt2seq_pad uk: ', testing_txt2seq_pad_uk)
print('='*100)
print('Shape dk: ', testing_txt2seq_pad_dk.shape)
print('Shape uk: ', testing_txt2seq_pad_uk.shape)
#testing_txt2seq_reshaped = testing_txt2seq_pad.reshape((-1, testing_txt2seq_pad[-1]))
print('='*100)

testing_txt2seq_reshaped_dk = testing_txt2seq_pad_dk.reshape(*testing_txt2seq_pad_dk.shape, 1)
testing_txt2seq_reshaped_uk = testing_txt2seq_pad_uk.reshape(*testing_txt2seq_pad_uk.shape, 1)

print('testing_txt2seq_reshaped: ', testing_txt2seq_reshaped_dk.shape)
print('testing_txt2seq_reshaped_uk: ', testing_txt2seq_reshaped_uk.shape)
print('='*100)

#======================================================================================

In [ ]:
import tensorflow as tf

dims2 = encdec_model_dev.predict([testing_txt2seq_reshaped_uk, testing_txt2seq_reshaped_dk]) #<---- What is the input UK , DK ???
#print(dims2)

predict1 = tf.argmax(dims2[0])
print('Predict 1:\n', predict1)

print('dims2.max', dims2.max())
print('dims2.min', dims2.min())
print('dims2.mean', dims2.mean())
print('dims2.ndim', dims2.ndim)
print('dims2.mean', dims2.min())
print('dims2.mean', dims2.shape)

print('='*100)
arrayout = np.argmax(dims2)
print('Array out:', arrayout)
print('='*100)
prediction_dk = np.argmax(dims2, axis = 0)
prediction_uk = np.argmax(dims2, axis = 1)
#==============================================================================

#print('prediction_dk: ', prediction_dk)
#print('prediction_uk: ', prediction_uk)
print('prediction_uk: \n', prediction_test)

print('='*100)
prediction_trans_dk = danish_fit2text.sequences_to_texts(prediction_dk)
prediction_trans_uk = english_fit2text.sequences_to_texts(prediction_uk)

print('Input DK: \n', text_testing_dk)
print('prediction_trans_dk: \n', prediction_trans_dk)
print('InputUK: \n', text_testing_uk)
print('prediction_trans_uk: \n', prediction_trans_uk)
print('='*100)

#==============================================================================